In [1]:
# the capital of france is paris

prompt = "the capital of the united states is not "

next_token = " "

output = "the capital of france is paris, "

girdi = "the letter capital of the united states is not"
cikti = "letter capital of the united states is not london"

In [9]:
from my_tokenizer import Tokenizer
from text_dataset import create_data_loader

def tokenize_and_save(input_file="text.txt", output_file="token_ids.txt", tokenizer_file="tokenizer.json"):
    """Tokenize text and save token IDs to file"""
    # Initialize tokenizer
    tokenizer = Tokenizer(tokenizer_file)
    
    # Read text
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()
    
    # Tokenize
    token_ids = tokenizer.encode(text)
    
    # Convert to Python integers if they're tensors
    if hasattr(token_ids, 'tolist'):  # If it's a tensor
        token_ids = token_ids.tolist()
    elif hasattr(token_ids[0], 'item'):  # If elements are tensors
        token_ids = [t.item() if hasattr(t, 'item') else int(t) for t in token_ids]
    
    # Save token IDs (space-separated integers only)
    with open(output_file, "w") as f:
        ids_text = " ".join(str(int(token_id)) for token_id in token_ids)
        f.write(ids_text)
    
    print(f"{len(token_ids)} token başarıyla kaydedildi!")
    print(f"İlk 10 token: {token_ids[:10]}")
    print(f"Son 10 token: {token_ids[-10:]}")
    
    return token_ids

def load_token_ids(file_path="token_ids.txt"):
    """Load token IDs from file with error handling"""
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read().strip()
    
    print(f"File content preview: {content[:200]}...")
    
    # Handle different formats that might be in the file
    token_ids = []
    
    # Split by whitespace and clean up
    tokens = content.split()
    
    for token in tokens:
        token = token.strip()
        if not token:
            continue
            
        # Skip tensor representations
        if token.startswith('tensor(') or token == 'tensor(0)':
            continue
            
        # Remove brackets, commas, and other punctuation
        token = token.replace('[', '').replace(']', '').replace(',', '').replace('(', '').replace(')', '')
        
        # Try to convert to int
        try:
            if token.isdigit() or (token.startswith('-') and token[1:].isdigit()):
                token_ids.append(int(token))
        except ValueError:
            print(f"Skipping invalid token: '{token}'")
            continue
    
    print(f"Successfully parsed {len(token_ids)} tokens")
    return token_ids

def test_dataloader(token_ids, context_length=8, stride=12, batch_size=1):
    """Test DataLoader functionality"""
    # Create DataLoader
    loader = create_data_loader(
        token_ids, 
        context_length, 
        stride, 
        batch_size=batch_size, 
        shuffle=False
    )
    
    print(f"DataLoader bilgileri:")
    print(f"  - Toplam token sayısı: {len(token_ids)}")
    print(f"  - Toplam batch sayısı: {len(loader)}")
    print(f"  - Context length: {context_length}")
    print(f"  - Stride: {stride}")
    print(f"  - Batch size: {batch_size}")
    print("-" * 50)
    
    # Show first few batches
    for i, (inp, tgt) in enumerate(loader):
        print(f"Batch {i+1}:")
        print(f"  Input:  {inp.squeeze().tolist()}")
        print(f"  Target: {tgt.squeeze().tolist()}")
        print()
        
        if i >= 2:  # Show first 3 batches
            break
    
    print("DataLoader testi tamamlandı!")
    return loader

        
def main():
    """Main execution function"""
    # Step 1: Tokenize and save (if no existing valid tokens)
    print("\n=== TOKENIZATION ===")
    token_ids = tokenize_and_save()
        
    print("\n=== DATALOADER TEST ===")
    # Step 2: Load token IDs and test DataLoader
    token_ids = load_token_ids()
    loader = test_dataloader(token_ids)  
       
    # Use the tokens we have
    final_tokens = load_token_ids()
    
    # Additional statistics
    print(f"\n=== STATISTICS ===")
    print(f"Toplam token sayısı: {len(final_tokens)}")
    print(f"Unique token sayısı: {len(set(final_tokens))}")
    if final_tokens:
        print(f"En sık kullanılan token: {max(set(final_tokens), key=final_tokens.count)}")
        print(f"Token değer aralığı: {min(final_tokens)} - {max(final_tokens)}")

if __name__ == "__main__":
    main()


=== TOKENIZATION ===
20721 token başarıyla kaydedildi!
İlk 10 token: [0, 61, 1, 61, 2, 61, 0, 61, 3, 61]
Son 10 token: [60, 61, 6, 61, 35, 61, 36, 61, 37, 59]

=== DATALOADER TEST ===
File content preview: 0 61 1 61 2 61 0 61 3 61 4 58 61 5 61 6 61 7 59 61 0 61 1 61 2 61 8 61 5 61 9 60 61 10 61 11 61 5 61 0 61 1 61 2 61 12 59 61 13 61 5 61 14 61 15 60 61 16 61 5 61 14 61 17 60 61 10 61 18 61 5 61 14 61 ...
Successfully parsed 20721 tokens
DataLoader bilgileri:
  - Toplam token sayısı: 20721
  - Toplam batch sayısı: 1727
  - Context length: 8
  - Stride: 12
  - Batch size: 1
--------------------------------------------------
Batch 1:
  Input:  [0, 61, 1, 61, 2, 61, 0, 61]
  Target: [61, 1, 61, 2, 61, 0, 61, 3]

Batch 2:
  Input:  [61, 5, 61, 6, 61, 7, 59, 61]
  Target: [5, 61, 6, 61, 7, 59, 61, 0]

Batch 3:
  Input:  [2, 61, 8, 61, 5, 61, 9, 60]
  Target: [61, 8, 61, 5, 61, 9, 60, 61]

DataLoader testi tamamlandı!
File content preview: 0 61 1 61 2 61 0 61 3 61 4 58 61 5 61 6 61 7 59 61

In [ ]:
data_iter = iter(loader)

next(data_iter)

In [10]:
##Embeddding,sözlük anlamlarının sayısal değerlerle ifade edilmesi

the_1 = 0.1
the_2 = 0.4
the_3 = 0.0
the_4 = 0.0

capital_1 = 0.0
capital_2 = 0.2
capital_3 = 0.24
capital_4 = 0.3

of_1 = 0.10
of_2 = 0.04
of_3 = 0.02
of_4 = 0.01

united_1 = 0.30
united_2 = 0.06
united_3 = 0.03
united_4 = 0.01

dict_v1 = [
  [0.1, 0.0, 0.0, 0.0],
  [0.4, 0.2, 0.04, 0.3],
  [0.0, 0.24, 0.02, 0.01],
  [0.0, 0.0, 0.03, 0.01],
  [0.0, 0.0, 0.0, 0.0]
]

dict_v1[0], dict_v1[1], dict_v1[2], dict_v1[3]


dict_v2 = {
  "the": [0.1, 0.0, 0.0, 0.0],
  "capital": [0.4, 0.2, 0.04, 0.3],
  "of": [0.0, 0.24, 0.02, 0.01],
  "united": [0.0, 0.0, 0.03, 0.01],
  "states": [0.0, 0.0, 0.0, 0.0]
}

dict_v2["the"], dict_v2["capital"], dict_v2["of"], dict_v2["united"], dict_v2["states"]

([0.1, 0.0, 0.0, 0.0],
 [0.4, 0.2, 0.04, 0.3],
 [0.0, 0.24, 0.02, 0.01],
 [0.0, 0.0, 0.03, 0.01],
 [0.0, 0.0, 0.0, 0.0])

In [11]:
import plotly.graph_objects as go
import plotly.offline

def plot_dots(sentences_data, title, dims=[0, 1, 2]):
  data = [
    go.Scatter3d(
      x=sentence_data["words"][:, dims[0]],
      y=sentence_data["words"][:, dims[1]],
      z=sentence_data["words"][:, dims[2]],
      mode="markers+text",
      marker=dict(
        size=6,
        color=sentence_data["color"],
      ),
      text=sentence_data["labels"],
      hoverinfo="text",
    ) for sentence_data in sentences_data
  ]

  layout = go.Layout(
    scene=dict(
      xaxis_title="Sertlik",
      yaxis_title="Parlaklık",
      zaxis_title="Kırmızılık",
    ),
    title=title,
  )

  fig = go.Figure(data=data, layout=layout)
  plotly.offline.iplot(fig)

In [12]:
import numpy as np

sentences = [
  {
    "words": np.array([
      [0.1, 0.20, 0.23, 0.0],
      [0.4, 0.2, 0.04, 0.3],
      [0.12, 0.24, 0.2, 0.01],
      [0.0, 0.0, 0.3, 0.01],
      [0.0, 0.0, 0.0, 0.0]
    ]),
    "labels": ["the", "capital", "of", "united", "states"],
    "color": "red",
  },
  {
    "words": np.array([
      [0.4, 0.2, 0.04, 0.3],
      [0.12, 0.24, 0.2, 0.01],
      [0.0, 0.0, 0.3, 0.01],
      [0.0, 0.0, 0.0, 0.0]
    ]),
    "labels": ["capital", "of", "united", "states"],
    "color": "blue",
  }
]

plot_dots(sentences, "Sözlük V1")

In [13]:
import torch

embeddings = torch.nn.Embedding(num_embeddings=64, embedding_dim=4)
sentence = "the capital of united states and the capital of france"
tokens = tokenizer.encode(sentence)
tokens

tensor([ 0, 61,  1, 61,  2, 61,  3, 61,  4, 58, 61, 10, 61,  0, 61,  1, 61,  2,
        61,  8])

In [14]:
meanings = embeddings(torch.tensor(tokens))
meanings

C:\Users\Zeynep\AppData\Local\Temp\ipykernel_22828\2280836142.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



tensor([[ 0.3002, -0.0116, -0.7680, -0.0361],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [ 0.2704,  0.5017, -0.7468,  1.0453],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [-0.7382, -1.5580, -0.7419,  0.1172],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [ 0.5411, -1.4492, -1.7196, -0.8313],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [-0.3930,  0.4900,  0.2360, -0.6421],
        [ 0.1779, -0.7912,  1.2710, -2.1091],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [ 1.1873, -1.0225,  0.4022, -0.1532],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [ 0.3002, -0.0116, -0.7680, -0.0361],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [ 0.2704,  0.5017, -0.7468,  1.0453],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [-0.7382, -1.5580, -0.7419,  0.1172],
        [-1.0889,  0.3013, -0.6797, -2.0736],
        [-0.1073, -1.3767, -0.6210, -0.4371]], grad_fn=<EmbeddingBackward0>)

In [15]:
sentences = [
  {
    "words": meanings.detach().numpy(),
    "labels": tokenizer.tokenize(sentence),
    "color": "red",
  },
]
plot_dots(sentences, "Sözlük")

In [16]:
def plot_dots(sentences_data, title, dims=[0, 1, 2]):
    data = []
    for sentence_data in sentences_data:
        scatter = go.Scatter3d(
            x=sentence_data["words"][:, dims[0]],
            y=sentence_data["words"][:, dims[1]], 
            z=sentence_data["words"][:, dims[2]],
            mode="markers+text",
            marker=dict(size=8, color=sentence_data["color"]),
            text=sentence_data["labels"],
            textposition="middle center"
        )
        data.append(scatter)
    
    fig = go.Figure(data=data)
    fig.update_layout(title=title)
    fig.show()
    return fig

In [17]:
from transformers import AutoTokenizer, Gemma3ForCausalLM
import torch
import plotly.graph_objects as go
import plotly.offline

gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
gemma_model = Gemma3ForCausalLM.from_pretrained("google/gemma-3-1b-it")

gemma_tokenizer.encode(sentence)

gemma_tokens = gemma_tokenizer.encode(sentence)
gemma_tokens

gemma_model.model.embed_tokens(torch.tensor(gemma_tokens))

gemma_meanings = gemma_model.model.embed_tokens(torch.tensor(gemma_tokens))
gemma_meanings.shape

gemma_tokenizer.tokenize(sentence)

gemma_sentences = [
  {
    "words": gemma_meanings.detach().numpy(),
    "labels": gemma_tokenizer.tokenize(sentence),
    "color": "red",
  },
]
plot_dots(gemma_sentences, "Gemma", dims=[20, 21, 22])


c:\Users\Zeynep\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

